In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import cv2 
import warnings

# Load data

In [ ]:
df_training_data  = pd.read_csv('../rsna-2024-lumbar-spine-degenerative-classification/train.csv')
df_training_data_label_co = pd.read_csv("../rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv")
df_training_data_description = pd.read_csv("../rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv")

# Visualise distribution 

In [ ]:
figure, axis = plt.subplots(1,3, figsize=(20,5)) 
for idx, d in enumerate(['foraminal', 'subarticular', 'canal']):
    diagnosis = list(filter(lambda x: x.find(d) > -1, df_training_data.columns))
    dff = df_training_data[diagnosis]
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        value_counts = dff.apply(pd.value_counts).fillna(0).T
    value_counts.plot(kind='bar', stacked=True, ax=axis[idx])
    axis[idx].set_title(f'{d} distribution')

# Load image by patient

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
import pydicom

class MRILoader(Dataset):
    def __init__(self, annotations, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = pydicom.dcmread(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
def display_coor_on_img(c, i):
    center_coordinates = (int(c['x']), int(c['y']))
    radius = 10
    color = (255, 0, 0)  # Red color in BGR
    thickness = 2
    IMG = i['dicom'].pixel_array
    IMG_normalized = cv2.normalize(IMG, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    IMG_with_circle = cv2.circle(IMG_normalized.copy(), center_coordinates, radius, color, thickness)
    
    # Convert the image from BGR to RGB for correct color display in matplotlib
    IMG_with_circle = cv2.cvtColor(IMG_with_circle, cv2.COLOR_BGR2RGB)
    
    return IMG_with_circle

In [ ]:
# Function to display images
def display_images(images, title, max_images_per_row=4):

    # Calculate the number of rows needed
    num_images = len(images)
    num_rows = (num_images + max_images_per_row - 1) // max_images_per_row  # Ceiling division

    # Create a subplot grid
    fig, axes = plt.subplots(num_rows, max_images_per_row, figsize=(5, 1.5 * num_rows))
    
    # Flatten axes array for easier looping if there are multiple rows
    if num_rows > 1:
        axes = axes.flatten()
    else:
        axes = [axes]  # Make it iterable for consistency

    # Plot each image
    for idx, image in enumerate(images):
        ax = axes[idx]
        #image = display_coor_on_img(coord[idx], image)
        ax.imshow(image, cmap='gray')  # Assuming grayscale for simplicity, change cmap as needed
        ax.axis('off')  # Hide axes

    # Turn off unused subplots
    for idx in range(num_images, len(axes)):
        axes[idx].axis('off')
    fig.suptitle(title, fontsize=16)

    plt.tight_layout()

In [ ]:
def load_by_patient_obj(path_dir):
    images_dic = {}
    axes_path = [x for x in os.listdir(path_dir) if not "." in x]
    for axe_path in axes_path :
        images_name = os.listdir(os.path.join(path_dir,axe_path))  
        images = []
        for image_name in images_name :
            image = pydicom.dcmread(os.path.join(os.path.join(path_dir,axe_path),image_name))
            images.append(image)
        images_dic.update({axe_path :images })
    return images_dic

In [ ]:
images_dic = load_by_patient_obj(os.path.join("../rsna-2024-lumbar-spine-degenerative-classification/train_images",str(4003253)))

In [ ]:
df_data = df_training_data_description.merge(df_training_data,on="study_id")

In [ ]:
for patient_id in df_data.study_id.unique():
    folders = df_data[df_data["study_id"] == patient_id].series_id.values
    for slicing in folders :
        display_images(
            [images_dic[str(slicing)][i].pixel_array for i in range(len(images_dic[str(slicing)]))],
            df_data[
                    (df_data["study_id"] == patient_id) & (df_data["series_id"] == slicing)
                ].series_description.values[0],
                max_images_per_row=4,

        )
    break

In [ ]:
df_training_data_label_co[(df_training_data_label_co["study_id"]==4003253)&(df_training_data_label_co["series_id"]==2448190387)]


In [ ]:
for patient_id in df_data.study_id.unique():
    folders = df_data[df_data["study_id"] == patient_id].series_id.values
    df_training_data_label_co[(df_training_data_label_co["study_id"]==4003253)&(df_training_data_label_co["series_id"]==702807833)]

In [ ]:
instances = df_training_data_label_co[(df_training_data_label_co["study_id"]==4003253)&(df_training_data_label_co["series_id"]==702807833)].instance_number.unique()

In [ ]:
for instance in instances :
    